In [10]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory, to_categorical
import os, glob
from tifffile import imread, imwrite
import numpy as np
import matplotlib.pyplot as plt


import pandas as pd 
labels = pd.read_pickle(f'{os.environ["HOME"]}/Dokumente/AML Projekt/labels_pkl/labels.pkl')
for i in range(len(labels)): ##über len(labels)
    im_old = imread(f'{os.environ["HOME"]}/Dokumente/AML Projekt/cropped_labels/{labels["img_folder"][i]}/{os.path.splitext(labels["img_name"][i])[0]}.tif')
    im_new = to_categorical(im_old)
    imwrite(f'{os.environ["HOME"]}/Dokumente/AML Projekt/categorical_labels/{labels["img_folder"][i]}/{os.path.splitext(labels["img_name"][i])[0]}.tif',im_new)

In [11]:
epochs = 100
num_batches = 250
batch_size = 8

patch_size = 800

num_classes = 44
classes = np.arange(44)
scores = [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 24., 25., 26.,
       27., 28., 30., 32., 33., 34., 36., 38., 39., 40., 42., 45., 48.,
       50., 51., 54., 57., 60.]

class MyLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, initial_learning_rate):
    self.initial_learning_rate = initial_learning_rate

  def __call__(self, epoch):
     return (self.initial_learning_rate - epoch/epochs)**0.9

def DiceLoss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = keras.backend.flatten(inputs)
    targets = keras.backend.flatten(targets)
    
    intersection = keras.backend.sum(keras.backend.dot(targets, inputs))
    dice = (2*intersection + smooth) / (keras.backend.sum(targets) + keras.backend.sum(inputs) + smooth)
    return 1 - dice + keras.losses.CategoricalCrossentropy()(targets,inputs)

loss = DiceLoss

optimizer = keras.optimizers.SGD(MyLRSchedule(1.),momentum=0.99,nesterov=True, name='SGD')



In [12]:
def load_label(path_to_label):

    label = imread(path_to_label)
    
    for i, score in enumerate(scores):
        label[label == score] = i

    label = to_categorical(label,num_classes)

    return label

In [13]:
def load_training_data(img_dir, label_dir):
    
    # get filenames
    img_names = sorted(glob.glob(img_dir+'/**/*'+'.JPG', recursive=True))
    label_names = sorted(glob.glob(label_dir+'/**/*'+'.tif', recursive=True))


    # load first label
    labels = np.zeros((batch_size,patch_size,patch_size,num_classes))

    # Initalise mini batch
    images = np.zeros((batch_size,patch_size,patch_size,3))

    for i, (img_name, label_name) in enumerate(zip(img_names[0:batch_size], label_names[0:batch_size])):

        # append label
        labels[i] = load_label(label_name)

        # append image
        new_img = plt.imread(img_name)
        new_img = new_img.astype(np.float32)
        new_img -= np.amin(new_img)
        new_img /= np.amax(new_img)
        images[i] = new_img
        

    return images, labels

In [14]:
#load the data to train and validate

train_img_path = 'T:/deepdart_data/cropped_images/800/d1_02_04_2020'
train_label_path = 'T:/deepdart_data/cropped_labels/d1_02_04_2020'

val_img_path = 'T:/deepdart_data/cropped_images/800/d1_02_06_2020'
val_label_path = 'T:/deepdart_data/cropped_labels/d1_02_06_2020'

x_train, y_train = load_training_data(train_img_path,train_label_path)
x_val, y_val = load_training_data(val_img_path,val_label_path)

In [15]:
y_train.shape

(8, 800, 800, 44)

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5966480380558612151
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5738397696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10711905365787207036
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [17]:
from keras.layers import MaxPooling2D, Conv2D, Conv2DTranspose, concatenate ,BatchNormalization, LeakyReLU

Norm = BatchNormalization
LReLU = LeakyReLU
Pool = MaxPooling2D


def NN_UNet():
    inputs = keras.Input(shape=(patch_size,patch_size,3))

    #Downsampeling
    x = Conv2D(32,3,padding='same')(inputs)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(32,3,padding='same')(x)
    x = Norm()(x)
    l1 = LReLU(alpha= 0.01)(x)

    x = Pool(pool_size= 2, strides= 2, padding='valid')(l1)

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    l2 = LReLU(alpha= 0.01)(x)

    x = Pool(pool_size= 2, strides= 2, padding='valid')(l2)

    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    l3 = LReLU(alpha= 0.01)(x)
    
    x = Pool(pool_size= 2, strides= 2, padding='valid')(l3)

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    l4 = LReLU(alpha= 0.01)(x)
    
    x = Pool(pool_size= 2, strides= 2, padding='valid')(l4)
    
    #Bottleneck
    x = Conv2D(512,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(512,3,padding='same')(x)
    x = Norm()(x)
    l5 = LReLU(alpha= 0.01)(x)
    
    #Upsampleing
    x = Conv2DTranspose(256,3,2,padding='same')(l5)
    x = concatenate([x,l4])

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(256,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)
    

    x = Conv2DTranspose(128,3,2,padding='same')(x)
    x = concatenate([x,l3])
    
    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(128,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)
    
    x = Conv2DTranspose(64,3,2,padding='same')(x)
    x = concatenate([x,l2])

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(64,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2DTranspose(32,3,2,padding='same')(x)
    x = concatenate([x,l1])

    x = Conv2D(32,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(32,3,padding='same')(x)
    x = Norm()(x)
    x = LReLU(alpha= 0.01)(x)

    x = Conv2D(44,1)(x)
    outputs = keras.activations.softmax(x)

    model = keras.Model(inputs,outputs,name='U-Net')
    return model

UNet = NN_UNet()
UNet.summary()


Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task

In [18]:
UNet.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    optimizer = optimizer,
    metrics=["accuracy"]    
)

history = UNet.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_val,y_val))
UNet.save("unet")
#test_scores = model.evaluate(x_test,y_test,verbose=2) 

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /jo

ResourceExhaustedError: Graph execution error:

Detected at node 'U-Net/leaky_re_lu_19/LeakyRelu' defined at (most recent call last):
    File "c:\Users\nwuen\.conda\envs\tensorflow\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\nwuen\.conda\envs\tensorflow\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\nwuen\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\nwuen\.conda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\nwuen\.conda\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\nwuen\.conda\envs\tensorflow\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\nwuen\.

In [16]:
test = UNet.predict(x_train)
test[0]

1/1 [==============================] - 1s 1s/step


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.